---
title: "25-因果推断中的线性回归"
author: "Simonzhou"
date: "2025-07-29"
date-modified: today
format: 
    html:
        code-fold: true
        code-line-numbers: true
        code-highlight: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 3
---

In [2]:
import stata_setup
stata_setup.config('C:/Program Files/Stata18', 'mp', splash=False)

# 线性回归

这里将使用一个案例演示 regress 和 teffects ra 的用法。

案例数据来自 “中国家庭追踪数据”（China Family Panel Studies, CFPS）2010年的数据，该数据集由中国社会科学院和北京大学共同开发，旨在收集中国家庭的社会、经济和人口信息。

数据总共包括 4137 名年龄在25-55 岁之间的成年人，集体如下：

In [3]:
%%stata
use "../../Data/causal-inference/cfps2010.dta", clear
* 查看数据随机十个数据
list in 1/10


. use "../../Data/causal-inference/cfps2010.dta", clear

. * 查看数据随机十个数据
. list in 1/10

     +------------------------------------------------------------------------+
  1. |       pid | provcd | gender | age | age2 |   age3 |    lninc | college |
     | 110003101 | 北京市 |     男 |  39 | 1521 |  59319 | 11.51293 |      否 |
     |------------------------------------------------------------------------|
     |        hukou      |     sibling      |     race      |      fmedu      |
     |     农村户口      |          否      |       是      |       缺失      |
     +------------------------------------------------------------------------+

     +------------------------------------------------------------------------+
  2. |       pid | provcd | gender | age | age2 |   age3 |    lninc | college |
     | 110005102 | 北京市 |     男 |  48 | 2304 | 110592 | 9.392662 |      否 |
     |------------------------------------------------------------------------|
     |        hukou      |     sibling      |   

## 简要的查看一下数据

In [4]:
%%stata
describe


Contains data from ../../Data/causal-inference/cfps2010.dta
 Observations:         4,137                  
    Variables:            12                  5 Aug 2022 19:25
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
pid             double  %12.0g     pid        个人id
provcd          double  %24.0g     provcd     省国标码
gender          double  %12.0g     gender     性别
age             float   %9.0g                 年龄
age2            float   %9.0g                 年龄平方
age3            float   %9.0g                 年龄三次方
lninc           float   %9.0g                 收入对数
college         double  %9.0g      yesorno    是否上大学
hukou           double  %12.0g     hukou      3岁时户口性质
sibling         float   %9.0g      yesorno    是否独生子女
race            double  %9.0g      yesorno    是否汉族
fme

分析变量为收入对数 `lninc`，核心自变量为是否上大学 `college`，该变量取值为 1 表示研究对象读过大学，取值为 0 表示没有读过大学。

基于上述两个变量，可以进行一元线性回归，具体如下：

In [5]:
%%stata
regress lninc college


      Source |       SS           df       MS      Number of obs   =     4,137
-------------+----------------------------------   F(1, 4135)      =    508.22
       Model |  671.883317         1  671.883317   Prob > F        =    0.0000
    Residual |  5466.56371     4,135  1.32202266   R-squared       =    0.1095
-------------+----------------------------------   Adj R-squared   =    0.1092
       Total |  6138.44703     4,136  1.48415063   Root MSE        =    1.1498

------------------------------------------------------------------------------
       lninc | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .823612   .0365338    22.54   0.000     .7519861     .895238
       _cons |   9.353189   .0230235   406.25   0.000     9.308051    9.398327
------------------------------------------------------------------------------


从分析结果可以看出，在不控制任何变量的情况下，college 的回归系数为 0.824 ，其标准误为 0.037。

用回归系数除以标准误即可得到 t 统计量的值，为 22.54，它的 P 值在小数点后三位都是 0，说明 college 对 lninc 的影响是非常显著的。

除此之外，Stata 还报告了置信度为 95% 的置信区间为 \[0.752, 0.896\]。

需要注意的是，上述标准误以及统计检验结果都是基于独立同分布假定计算的，因此在实际应用中需要谨慎解读。

在显示条件下，该假定通常不能得到满足，因此，建议使用更加稳健的标准误和统计检验。

## 使用稳健标准误

使用怀特异方差稳健标准误来调整异方差的影响，使用方法在 `regress` 命令后使用选项 `vce(robust)`。

In [6]:
%%stata
regress lninc college, vce(robust)


Linear regression                               Number of obs     =      4,137
                                                F(1, 4135)        =     582.05
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1095
                                                Root MSE          =     1.1498

------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .823612   .0341385    24.13   0.000     .7566823    .8905418
       _cons |   9.353189   .0256882   364.10   0.000     9.302826    9.403552
------------------------------------------------------------------------------


从结果中可以看到，使用稳健标准误后，college 的回归系数仍然为 0.824，但其标准误变为 0.034，相比之前略有下降，t 统计量变为 24.13，P 值在小数点后三位都是 0，说明 college 对 lninc 的影响仍然是非常显著的。

由于异方差条件下的标准误和统计检验结果更加可信，因此建议用户在使用 `regress` 命令时，使用选项 `vce(robust)`。

## 多阶段整群抽样

用户还可以使用选项 vce(cluster clustvar) 进一步放松误差项之间相互独立的假定。

CFPS 采用的是以地理区域为抽样单位的多阶段整群抽样，这样会导致同一个抽样单位中的样本存在自相关。

为了降低自相关的影响，以调查样本所在省（provcd）为聚类变量，使用选项 `vce(cluster provcd)`，计算聚类稳健标准误，命令和结果如下：

In [7]:
%%stata
regress lninc college, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(1, 24)          =     271.17
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1095
                                                Root MSE          =     1.1498

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .823612   .0500155    16.47   0.000     .7203851     .926839
       _cons |   9.353189   .1084703    86.23   0.000     9.129317    9.577061
------------------------------------------------------------------------------


在使用 `vce(cluster provcd)` 后，college 的回归系数仍然为 0.824，但其标准误变为 0.050，相比之前略有上升，t 统计量变为 16.47，P 值在小数点后三位都是 0，说明 college 对 lninc 的影响仍然是非常显著的。

在这个聚类稳健标准误的结果中，统计检验结果没有受到影响，相比之前，使用聚类稳健标准误的结果最为稳健，因此，后续都将使用此类标准误的分析结果。

上述分析都只包含 college 这一个自变量，在这种情况下，进行因果推断必须满足严格的可忽略性假定。即要求大学学历在研究对象中随机分配，这个假定显然与中国的实际情况相差很远，因此，上述分析结果仅能代表上大学的人和不上大学的人在收入对数上的平均差异，不能解释为大学对收入的因果影响。

## 控制变量

为了使回归分析结果具有因果解释的效应，需要在模型中纳入控制变量，以尽可能地满足条件独立假定。

这里我们认为，就这个研究问题来说，研究对象出生时的户口性质（hukou）是一个非常重要的混杂变量。

一方面，由于教育资源分布的城乡不均，城市人比农村人拥有更多考上大学的机会；另一方面，因为劳动力市场上的城乡分割，城市人也比农村人拥有更好的收入前景。

综合这两个方面，有必要在模型分析时控制 hukou 变量。

In [8]:
%%stata
regress lninc college hukou, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(2, 24)          =     219.69
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1169
                                                Root MSE          =     1.1451

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .798281   .0451727    17.67   0.000     .7050491     .891513
       hukou |   .2155333   .0611855     3.52   0.002     .0892526    .3418141
       _cons |    9.27541   .0993474    93.36   0.000     9.070367    9.480453
--------------

从上述多元线性回归分析结果可以发现，在控制了 hukou 变量后，college 的回归系数有所下降，变为 0.798，说明之前使用的一元线性回归确实高估了上大学对收入的影响。

## 偏回归系数

在控制 hukou 后，college 的回归系数将变为一个偏回归系数，它可以通过两步线性回归得到。

具体来说，第一步回归分析以 college 为因变量，hukou 为自变量，这一步回归分析的目的是获取 college 的残差。

In [9]:
%%stata
reg college hukou, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(1, 24)          =      27.84
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0137
                                                Root MSE          =     .48606

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
     college | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       hukou |    .116539   .0220861     5.28   0.000     .0709555    .1621224
       _cons |   .3496532   .0161816    21.61   0.000     .3162561    .3830504
------------------------------------------------------------------------------


接着，使用 `predict` 命令并结合选项 `residuals` 获得回归的残差 $e$ 。

In [10]:
%%stata
predict e, residuals

再使用 lninc 作为因变量，$e$ 作为自变量进行第二步线性回归分析，这一步回归分析的结果就是 college 的偏回归系数。

In [11]:
%%stata
reg lninc e, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(1, 24)          =     321.05
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1014
                                                Root MSE          =      1.155

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
           e |    .798281    .044552    17.92   0.000     .7063302    .8902318
       _cons |   9.680285   .1232655    78.53   0.000     9.425877    9.934692
------------------------------------------------------------------------------


从这一步回归的分析结果来看，e 的回归系数与之前控制 hukou 以后得到的 college 对 lninc 的回归系数完全相同。

由此可知，通过两步法可以再现多元线性回归张的统计过程。

不过，通过两步法得到的回归系数的标准误是错误的，因为第二步回归无法自动对完整模型中的参数数量进行校正，因此，这里建议用户直接使用多元线性回归进行统计控制。

这里，使用两步法只是为了帮助用户更好地理解多元线性回归的统计控制和偏回归系数的含义。

## 对遗漏变量偏差公式进行检验

在多元线性回归中，遗漏变量偏差公式是一个重要的概念，它可以帮助我们理解控制变量对回归系数的影响。

遗漏变量偏差公式（omitted variable bias formula）为：

$$\tau '-\tau = \beta_k \gamma_D$$

根据这个公式，纳入 hukou 变量之后，college 的回归系数的变化量应等于 college 对 hukou 的回归系数与 hukou 与 lninc 的回归系数的乘积。

为了对上述关系进行检验，我们首先将控制 hukou 和不控制 hukou 时 college 的回归系数相减，得到二者的差异：

In [12]:
%%stata
dis .823612-.798281

.025331


接下来，以 hukou 为因变量，college 为自变量拟合回归方程，这样就可以得到 college 对 hukou 的回归系数。

然后，将该系数与多元线性回归中 hukou 对 lninc 的回归系数相乘，得到一个值。

In [13]:
%%stata
reg hukou college, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(1, 24)          =      33.17
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0137
                                                Root MSE          =     .48812

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       hukou | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |   .1175271   .0204066     5.76   0.000     .0754099    .1596443
       _cons |   .3608661   .0559186     6.45   0.000     .2454558    .4762764
------------------------------------------------------------------------------


In [14]:
%%stata
dis .1175271*.2155333

.025331


这个乘积与 college 的系数变化完全相同，也是 0.025.

这充分说明，遗漏变量偏差计算公式在实践中是完全正确的。

## 纳入更多的控制变量

在上大学对收入的影响这个例子中，hukou 虽然是一个重要的混杂变量，但很可能不是唯一的混杂变量。因此，有必要在回归方程中纳入更多的控制变量。

在回归方程包含多个控制变量的情况下， 必须考虑模型的设定问题。

以同时包含 hukou 和 age 两个控制变量为例，如何设定这两个变量的函数形式才能进行最充分的统计控制？

针对这个问题，最简单的答案就是进行饱和回归，即针对 hukou 和 age 的每个取值和所有交互项均设置一个虚拟变量。具体如下：

In [15]:
%%stata
reg lninc college hukou##i.age, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(23, 24)         =          .
                                                Prob > F          =          .
                                                R-squared         =     0.1501
                                                Root MSE          =     1.1316

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |   .7477749   .0544757    13.73   0.000     .6353425    .8602073
             |
       hukou |
   城镇户口  |  -.0787305   .1418798    -0.55   0.584    -.3715559    .2140949
             |
         age |
         26  |   .3980867   .1481415 

这样做的好处是不存在模型设定的偏差。

但如上所示，饱和回归也会出现待估参数过多的问题。特别是其中包含像 age 这样连续取值的变量时，这个问题会变得更加严重。

为了避免参数过多给模型估计和解释造成的困难，一个可行的解决方案是对控制变量影响因变量的方式进行参数限定，如使用幂函数来近似 age 对 lninc 的影响，不过需要对这种近似的效果进行评估。

为此，用户需要先计算饱和回归的模型拟合指标，具体如下：

In [16]:
%%stata
estat ic


Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |          N   ll(null)  ll(model)      df        AIC        BIC
-------------+---------------------------------------------------------------
           . |      4,137  -6686.381  -6349.885      24   12747.77   12899.63
-----------------------------------------------------------------------------
Note: BIC uses N = number of observations. See [R] IC note.


接下来，使用年龄的一次项（age）、二次项（age3）和三次项（age3）替代虚拟变量，进行线性回归分析，同时计算这个模型的拟合指标：

In [17]:
%%stata
reg lninc college hukou##(c.age c.age2 c.age3), vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(8, 24)          =      88.50
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1331
                                                Root MSE          =     1.1354

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |   .7547539   .0501103    15.06   0.000     .6513314    .8581763
             |
       hukou |
   城镇户口  |  -.5628695   3.164092    -0.18   0.860    -7.093234    5.967495
         age |   .6604027   .2081753     3.17   0.004       .23075 

In [20]:
%%stata
estat ic


Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |          N   ll(null)  ll(model)      df        AIC        BIC
-------------+---------------------------------------------------------------
           . |      4,137  -6686.381  -6397.289       6   12806.58   12844.54
-----------------------------------------------------------------------------
Note: BIC uses N = number of observations. See [R] IC note.


可以发现，这个模型和之前的饱和回归模型相比，参数数量大大减少，但模型的AIC和BIC却比饱和模型更低，因此，从模型拟合的角度来说，这个模型更加简约，所以可以用年龄的三次函数近似其对 lninc 的影响。

最后，进一步删除上述模型中 age、age2和age3 与 hukou 的交互项后再进行回归分析，同时计算模型拟合指标：

In [21]:
%%stata
reg lninc college hukou age age2 age3, vce(cluster provcd)


Linear regression                               Number of obs     =      4,137
                                                F(5, 24)          =     130.17
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1304
                                                Root MSE          =     1.1367

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |   .7483616   .0474037    15.79   0.000     .6505252    .8461981
       hukou |   .2233219   .0657661     3.40   0.002     .0875874    .3590565
         age |   .6884785    .143905     4.78   0.000     .3914732    .9854838
        age2 |

In [22]:
%%stata
estat ic


Akaike's information criterion and Bayesian information criterion

-----------------------------------------------------------------------------
       Model |          N   ll(null)  ll(model)      df        AIC        BIC
-------------+---------------------------------------------------------------
           . |      4,137  -6686.381  -6397.289       6   12806.58   12844.54
-----------------------------------------------------------------------------
Note: BIC uses N = number of observations. See [R] IC note.


可以发现，这个模型是三个模型中最简约的，同时其AIC和BIC也是三个模型中最小的。

这说明，只要在模型中纳入 hukou、age、age2 和 age3 这四个主效应项就可以很好近似 hukou 和 age 对 lninc 的影响。

而且，对比这个模型中 college 的回归系数与之前饱和回归中的系数，可以发现二者几乎不存在任何差异（若保留三位小数都是0.748），所以，从统计控制角度来说，这个更加简约的模型与之前的复杂模型几乎也是等价的。

## 纳入分类变量

接下来，我们又在模型中纳入了性别（gender）、民族（race）、是否独生子女（sibling）和父母是否读过高中（fmedu）这四个控制变量，他们都是分类变量，因此都以虚拟变量的形式纳入回归方程。具体如下：

In [23]:
%%stata
reg lninc college hukou age age2 age gender race sibling i.fmedu, vce(cluster provcd)

note: age omitted because of collinearity.

Linear regression                               Number of obs     =      4,137
                                                F(9, 24)          =     234.58
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1574
                                                Root MSE          =     1.1195

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |   .7445286   .0441545    16.86   0.000     .6533982     .835659
       hukou |   .2210093   .0577399     3.83   0.001     .1018401    .3401785
         age |   .1415978   .0245026     5.78   0.